In [2]:
import pandas as pd

In [1]:
import networkx as nx
G=nx.DiGraph() 

#Creating the graph

with open("wiki-topcats-reduced.txt") as f:
    for line in f:
        list1 = line.strip().split("\t")
        G.add_node(list1[0])
        G.add_edge(list1[0],list1[1])

### adding attribute 'Category' to each node of the graph

for i in G:
    G.node[i]['Category']=[]
    
from collections import defaultdict
category_dict = defaultdict()

with open("wiki-topcats-categories.txt", "r") as f:
    lst2=G.nodes()
    for line in f:
        cat_list = line.strip().split(";")
        category = cat_list[0][9:]
        lst = cat_list[1].strip().split(" ")
        if len(lst) > 3500:   
            
            lst1=[el for el in lst if el in lst2]
            category_dict[category] = lst1

#Assign attributes to each node
for cat in category_dict:
    lst = category_dict[cat]
    for e in lst:
        if e in G:
            G.node[e]['Category'].append(cat)

In [15]:
dfg=pd.read_csv('Graphdf.csv')
dfg1 = pd.read_csv('Graphdf1.csv')

In [31]:
dfg = dfg.drop(columns="Unnamed: 0")

dfg1 = dfg1.drop(columns="Unnamed: 0")

dfg = pd.concat([dfg,dfg1])

dfg=dfg.reset_index(drop=True)

dfg = dfg.rename(columns={"0": "Category", "1": "Distance"})

dfg = dfg.sort_values(by='Distance')

dfg = dfg.reset_index(drop=True)
dfg

,Category,Distance
0,Members_of_the_United_Kingdom_Parliament_for_E...,7.000000
1,Living_people,8.000000
2,Harvard_University_alumni,8.000000
3,Year_of_birth_missing_(living_people),8.000000
4,English_footballers,9.000000
5,Major_League_Baseball_pitchers,9.000000
6,English_cricketers,9.000000
7,Rivers_of_Romania,9.000000
8,The_Football_League_players,inf
9,Association_football_forwards,inf


Last point: We want to keep an information, score, into each node. first, we initialized every node as string, in order to make a double check (if we still didn't pass throught that node it will be still a string, but if there are no in edges this information will be 0). This initialization will be done into G, our original graph.

In [33]:
nodes_G = G.nodes()
for n in nodes_G:
    G.node[n]['score'] = ''

In [61]:
input_category='Indian_films'
c0 = category_dict[input_category][0:2000]    
sub = G.subgraph(c0)
for s in sub:
    G.node[s]['score'] = len(sub.in_edges(s))

In [62]:
categories = dfg['Category'].values #taking all categories from the ranking dataframe 

In [68]:
l = c0 #list of nodes in the subgraph (now only input_category nodes)
for c in categories:
    l = l + category_dict[c]
    sub1 = G.subgraph(l)
    for e1 in sub1: 
        if not isinstance(G.node[e1]['score'],str):
            continue
        else:
            ie1 = sub1.in_edges(e1)
            ie1 = [el[0] for el in ie1]
                   
            score = 0
            for f in ie1:
                
                if G.node[f]['score'] == '':
                    score +=1
                else:
                    score += G.node[f]['score']
            G.node[e1]['score'] = score
    

In [69]:
len(sub1)

390709

In [70]:
c0[0:10]

['29286',
 '53565',
 '70274',
 '70275',
 '123378',
 '123382',
 '134667',
 '134698',
 '134750',
 '134752']

In [60]:
sub.in_edges('70275')

InEdgeDataView([('70274', '70275')])

In [71]:
G.node['70275']['score']

1

In [72]:
G.in_edges('70275')

InEdgeDataView([('70274', '70275'), ('581966', '70275'), ('582017', '70275')])

In [73]:
G.in_edges('70274')

InEdgeDataView([('586702', '70274')])

In [77]:
for el in c0:
    if el == '582017':
        print('yes')